# Telco Customer Churn

In [44]:
# set task name
task_name = 'telco_churn-v2'

# load common
%run common.ipynb

Cortex Python SDK v6.2.0
Cortex Discovery Libraries v3.5.14


## Synthetic Build
Synthetic build uses the `SyntheticBuilder` capability

In [45]:
from ds_discovery import SyntheticBuilder, Transition

In [46]:
builder = SyntheticBuilder.from_env(task_name, has_contract=False)

In [47]:
# set the output
builder.set_persist(builder.pm.file_pattern(name='synthetic', prefix='Telco-Customer-Churn', file_type='parquet'))

# point to the clean sample
builder.add_connector_uri('clean_sample', uri=Transition.from_env(task_name).get_persist_contract().uri)
sample = builder.load_canonical('clean_sample')

In [48]:
# target
y = sample['Churn']
# features
X = sample.drop(['Churn', 'CustomerID'], axis=1)

In [49]:
# synthesis the sample
size = 10000
a ={}
for c in X.columns:
    a.update({c: {}})
    
# churn is the target
df = builder.tools.model_analysis(size, other='clean_sample', columns_list=[{'Churn':{}},a], column_name='analysis')
df.shape

(10000, 51)

In [50]:
# rebuild the customer_id as a unique reference
df['CustomerID'] = builder.tools.get_number(from_value=1000000, to_value=9999999, at_most=1, size=size, column_name='CustomerID')

In [51]:
# run the pipeline 
builder.run_component_pipeline()

In [53]:
df.head()

,Churn,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,ChurnRate,Churn_Score,CLTV,Churn_Reason,Count,Country,State,City,Zip_Code,Lat_Long,Latitude,Longitude,Age,Under_30,Married,Referred_a_Friend,Number_of_Referrals,Offer,Avg_Monthly_Long_Distance_Charges,Avg_Monthly_GB_Download,Streaming_Music,Premium_Tech_Support,Unlimited_Data,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Satisfaction_Score,Customer_Status,Churn_Score.1,Churn_Category,CustomerID
0,True,Female,0.670,False,True,31.758,False,No phone service,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Bank transfer (automatic),115.611,7807.132,0.000,17.766,2960.436,None,1.000,United States,California,Big Bend,90577.715,"38.759751, -121.288545",39.718,-114.918,76.602,True,False,False,7.879,Offer C,26.554,68.317,True,True,False,29.849,62.335,2938.501,11274.277,4.616,Joined,17.766,0,8267499
1,False,Female,0.255,False,False,12.783,True,Yes,Fiber optic,Yes,No,Yes,Yes,No,No,Two year,False,Credit card (automatic),78.684,472.693,0.000,54.240,4705.006,None,1.000,United States,California,Hornitos,93629.626,"38.377165, -123.037957",38.455,-123.267,46.954,False,False,False,3.475,Offer B,38.657,14.845,False,False,True,5.819,44.586,539.339,1831.558,3.730,Stayed,54.240,0,5117049
2,False,Female,0.688,False,True,36.654,False,No phone service,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Bank transfer (automatic),92.321,6820.774,0.000,22.226,2534.988,None,1.000,United States,California,Mendocino,90708.412,"34.244411, -117.072654",39.586,-116.949,70.360,True,False,False,8.236,Offer C,28.345,60.479,True,True,False,24.112,63.927,2471.889,10638.192,4.602,Joined,22.226,0,1202049
3,True,Male,0.184,True,False,13.052,True,No,Fiber optic,Yes,No,Yes,Yes,No,No,Two year,True,Electronic check,63.292,467.326,0.000,35.309,3959.168,None,1.000,United States,California,Lodi,93430.308,"37.106902, -122.100006",35.156,-118.069,45.780,False,True,True,3.102,None,47.690,22.309,False,False,True,8.278,31.517,995.135,2056.744,3.787,Stayed,35.309,0,6392799
4,False,Male,0.232,True,False,59.101,True,No,DSL,No,Yes,No,No,Yes,Yes,Month-to-month,True,Electronic check,22.633,210.424,0.000,68.311,5275.566,None,1.000,United States,California,Carson,94518.069,"34.165383, -118.123752",33.832,-119.560,21.300,False,True,True,1.746,None,10.216,10.933,False,False,True,14.670,34.150,1140.572,3462.656,3.571,Stayed,68.311,0,2408949
